# K214142070-NGUYỄN THỊ LINH

# import  các thư viện cần thiết và dữ liệu

In [36]:
import pandas as pd
import numpy as np
import xgboost as xgb
import plotly.graph_objects as go
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 20, 10
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0,1))

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense
from keras.models import Sequential
from keras.layers import Dense, GRU
import xgboost as xgb


In [37]:
#Import dữ liệu
df = pd.read_excel("E:\\stock.data.xlsx")

# Định dạng cột Date
df['Date'] = pd.to_datetime(df['Date'], format='%m/%d/%Y', errors='coerce')

#kiểm tra dữ liệu
print(df)

           Date     Open     High      Low    Close   Volume
0    2015-01-05  26610.0  26610.0  25280.0  25280.0       20
1    2015-01-06  26390.0  26390.0  25500.0  26160.0     7040
2    2015-01-07  26160.0  26160.0  25720.0  25940.0     3150
3    2015-01-08  25940.0  25940.0  25280.0  25940.0    11510
4    2015-01-09  25940.0  27270.0  25940.0  27270.0     2340
...         ...      ...      ...      ...      ...      ...
2341 2024-05-27  71000.0  71700.0  69600.0  71300.0   878800
2342 2024-05-28  72400.0  76000.0  71300.0  75200.0  3618500
2343 2024-05-29  75200.0  75300.0  73100.0  73800.0  1871800
2344 2024-05-30  73400.0  73600.0  70900.0  72400.0  2095500
2345 2024-05-31  72400.0  72500.0  71300.0  72500.0  1125800

[2346 rows x 6 columns]


In [38]:
#Mô tả dữ liệu
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2346 entries, 0 to 2345
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Date    2346 non-null   datetime64[ns]
 1   Open    2346 non-null   float64       
 2   High    2346 non-null   float64       
 3   Low     2346 non-null   float64       
 4   Close   2346 non-null   float64       
 5   Volume  2346 non-null   int64         
dtypes: datetime64[ns](1), float64(4), int64(1)
memory usage: 110.1 KB
None


# Biểu đồ giá giao dịch chứng khoán thực tế

In [39]:
# Tạo biểu đồ tương tác
fig = go.Figure()
# Thêm trace cho giá đóng cửa (Close)
fig.add_trace(go.Scatter(x=df['Date'], y=df['Close'], mode='lines', name='Giá đóng cửa (Close)'))
# Thêm trace cho khối lượng giao dịch (Volume)
fig.add_trace(go.Bar(x=df['Date'], y=df['Volume'], name='Khối lượng giao dịch (Volume)', yaxis='y2', opacity=0.6))

# Thiết lập các trục y
fig.update_layout(
    title='BIỂU ĐỒ GIÁ ĐÓNG CỬA VÀ KHỐI LƯỢNG GIAO DỊCH CHỨNG KHOÁN THEO NGÀY(CTD)',
    xaxis_title='Ngày',
    yaxis_title='Giá đóng cửa (Close)',
    yaxis=dict(
        title='Giá đóng cửa (Close)',
        showgrid=False  # Tắt grid lines cho yaxis
    ),
    yaxis2=dict(
        title='Khối lượng giao dịch (Volume)',
        overlaying='y',
        side='right',
        showgrid=False  # Tắt grid lines cho yaxis2
    ),
    xaxis=dict(
        showgrid=False  # Tắt grid lines cho xaxis
    ),
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    )
)
fig.show()


# TIẾN HÀNH HUẤN LUYỆN CÁC MÔ HÌNH DỰ ĐOÁN GIÁ CHỨNG KHOÁN

In [40]:
# Sắp xếp dữ liệu theo chỉ mục
data = df.sort_index(ascending=True, axis=0)

# Tạo DataFrame mới chỉ với cột 'Date' và 'Close'
new_dataset = pd.DataFrame(index=range(0, len(df)), columns=['Date', 'Close'])

for i in range(0, len(data)):
    new_dataset["Date"][i] = data['Date'][i]
    new_dataset["Close"][i] = data["Close"][i]

# Đặt 'Date' làm chỉ mục và loại bỏ cột 'Date'
new_dataset.index = new_dataset.Date
new_dataset.drop("Date", axis=1, inplace=True)

# Chuyển đổi DataFrame sang mảng numpy
final_dataset = new_dataset.values

# Chia dữ liệu thành tập train (70%) và test (30%)
train_size = int(len(final_dataset) * 0.7)
train_data = final_dataset[:train_size]
valid_data = final_dataset[train_size:]

# Chuẩn hóa dữ liệu
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(final_dataset)


# 1. MÔ HÌNH LSTM

In [41]:
# Chuẩn bị dữ liệu train
x_train_data, y_train_data = [], []

for i in range(60, len(train_data)):
    x_train_data.append(scaled_data[i-60:i, 0])
    y_train_data.append(scaled_data[i, 0])

x_train_data, y_train_data = np.array(x_train_data), np.array(y_train_data)
x_train_data = np.reshape(x_train_data, (x_train_data.shape[0], x_train_data.shape[1], 1))
# Xây dựng và huấn luyện mô hình LSTM
lstm_model = Sequential()
lstm_model.add(LSTM(units=50, return_sequences=True, input_shape=(x_train_data.shape[1], 1)))
lstm_model.add(LSTM(units=50))
lstm_model.add(Dense(1))

lstm_model.compile(loss='mean_squared_error', optimizer='adam')
lstm_model.fit(x_train_data, y_train_data, epochs=1, batch_size=1, verbose=2)

# Lấy mẫu test để dự đoán mô hình vừa tạo ở bước trên
inputs_data = new_dataset[len(new_dataset) - len(valid_data) - 60:].values
inputs_data = inputs_data.reshape(-1, 1)
inputs_data = scaler.transform(inputs_data)

X_test = []
for i in range(60, inputs_data.shape[0]):
    X_test.append(inputs_data[i-60:i, 0])
X_test = np.array(X_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

closing_price = lstm_model.predict(X_test)
closing_price = scaler.inverse_transform(closing_price)

# Lưu model lại
lstm_model.save("saved_model.h5")

# Giả sử new_dataset là DataFrame và closing_price là danh sách các dự đoán
# train_size là kích thước của tập dữ liệu huấn luyện
train_data = new_dataset[:train_size]
valid_data = new_dataset[train_size:].copy()
valid_data['Predictions'] = closing_price

# Tạo biểu đồ tương tác
fig = go.Figure()

# Thêm trace cho giá đóng cửa của tập huấn luyện
fig.add_trace(go.Scatter(x=train_data.index, y=train_data['Close'], mode='lines', name='Train Close Price'))

# Thêm trace cho giá đóng cửa thực tế của tập kiểm tra
fig.add_trace(go.Scatter(x=valid_data.index, y=valid_data['Close'], mode='lines', name='Test Close Price'))

# Thêm trace cho giá đóng cửa dự đoán của tập kiểm tra
fig.add_trace(go.Scatter(x=valid_data.index, y=valid_data['Predictions'], mode='lines', name='Predicted Close Price'))

# Thiết lập tiêu đề và trục
fig.update_layout(
    title='Biểu đồ Dự đoán giá đóng cửa chứng khoán bằng LSTM',
    xaxis_title='Ngày',
    yaxis_title='Giá Đóng Cửa (Close)',
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    )
)

# Hiển thị biểu đồ
fig.show()

C:\Users\USER\anaconda3\lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



1582/1582 - 25s - 16ms/step - loss: 0.0018
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step


In [42]:
# Đánh giá mô hình
mae = mean_absolute_error(valid_data['Close'], valid_data['Predictions'])
mse = mean_squared_error(valid_data['Close'], valid_data['Predictions'])
rmse = np.sqrt(mse)
r2 = r2_score(valid_data['Close'], valid_data['Predictions'])

print(f"Mean Absolute Error (MAE): {mae}")
print(f"Mean Squared Error (MSE): {mse}")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"R-squared (R²): {r2}")

Mean Absolute Error (MAE): 4172.814369895243
Mean Squared Error (MSE): 24122099.44014836
Root Mean Squared Error (RMSE): 4911.4253980029425
R-squared (R²): 0.8952000122585304


# 2.MÔ HÌNH GRU

In [43]:
# Tạo dataset để dự báo
X = []
y = []

for i in range(60, len(scaled_data)):
    X.append(scaled_data[i-60:i, 0])
    y.append(scaled_data[i, 0])

X, y = np.array(X), np.array(y)

# Chia dữ liệu thành tập train và test
train_size = int(len(X) * 0.7)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=(1 - train_size / len(X)), shuffle=False)

# Reshape lại dữ liệu để phù hợp với đầu vào của GRU
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

# Xây dựng mô hình GRU
model = Sequential()
model.add(GRU(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(GRU(units=50))
model.add(Dense(1))

model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(X_train, y_train, epochs=1, batch_size=1, verbose=2)

# Dự đoán giá đóng cửa
predictions = model.predict(X_test)
predictions = scaler.inverse_transform(predictions)

# Tạo chỉ mục cho valid_data
valid_index = new_dataset.index[len(new_dataset) - len(y_test):]

# Tạo DataFrame cho valid_data với các dự đoán
valid_data = pd.DataFrame(index=valid_index)
valid_data['Close'] = new_dataset['Close'][len(new_dataset) - len(y_test):]
valid_data['Predictions'] = predictions

# Tạo biểu đồ tương tác
fig = go.Figure()

# Thêm trace cho giá đóng cửa của tập huấn luyện
fig.add_trace(go.Scatter(x=new_dataset[:train_size].index, y=new_dataset[:train_size]['Close'], mode='lines', name='Train Close Price'))

# Thêm trace cho giá đóng cửa thực tế của tập kiểm tra
fig.add_trace(go.Scatter(x=valid_data.index, y=valid_data['Close'], mode='lines', name='Test Close Price'))

# Thêm trace cho giá đóng cửa dự đoán của tập kiểm tra
fig.add_trace(go.Scatter(x=valid_data.index, y=valid_data['Predictions'], mode='lines', name='Predicted Close Price'))

# Thiết lập tiêu đề và trục
fig.update_layout(
    title='Biểu đồ Dự đoán giá đóng cửa chứng khoán bằng GRU',
    xaxis_title='Ngày',
    yaxis_title='Giá Đóng Cửa (Close)',
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    )
)

# Hiển thị biểu đồ
fig.show()

C:\Users\USER\anaconda3\lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



1599/1599 - 26s - 16ms/step - loss: 0.0018
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step


In [47]:
# Đánh giá mô hình
mae = mean_absolute_error(valid_data['Close'], valid_data['Predictions'])
mse = mean_squared_error(valid_data['Close'], valid_data['Predictions'])
rmse = np.sqrt(mse)
r2 = r2_score(valid_data['Close'], valid_data['Predictions'])

print(f"Mean Absolute Error (MAE): {mae}")
print(f"Mean Squared Error (MSE): {mse}")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"R-squared (R²): {r2}")

Mean Absolute Error (MAE): 2401.8788039028386
Mean Squared Error (MSE): 13507435.881705053
Root Mean Squared Error (RMSE): 3675.2463702050036
R-squared (R²): 0.9426838569800213


# 3. MÔ HÌNH XGBoost

In [51]:


# Tạo dataset để dự báo
X = []
y = []

for i in range(60, len(scaled_data)):
    X.append(scaled_data[i-60:i, 0])
    y.append(scaled_data[i, 0])

X, y = np.array(X), np.array(y)

# Chia dữ liệu thành tập train và test
train_size = int(len(X) * 0.7)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=(1 - train_size / len(X)), shuffle=False)

# XGBoost yêu cầu dữ liệu đầu vào 2D, nên reshape lại
X_train_reshaped = X_train.reshape(X_train.shape[0], X_train.shape[1])
X_test_reshaped = X_test.reshape(X_test.shape[0], X_test.shape[1])

# Huấn luyện mô hình XGBoost
model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=1000, learning_rate=0.01)
model.fit(X_train_reshaped, y_train)

# Dự đoán giá đóng cửa
predictions = model.predict(X_test_reshaped)
predictions = scaler.inverse_transform(predictions.reshape(-1, 1))

# Tạo chỉ mục cho valid_data
valid_index = new_dataset.index[len(new_dataset) - len(y_test):]

# Tạo DataFrame cho valid_data với các dự đoán
valid_data = pd.DataFrame(index=valid_index)
valid_data['Close'] = new_dataset['Close'][len(new_dataset) - len(y_test):]
valid_data['Predictions'] = predictions

# Tạo biểu đồ tương tác
fig = go.Figure()

# Thêm trace cho giá đóng cửa của tập huấn luyện
fig.add_trace(go.Scatter(x=new_dataset[:train_size].index, y=new_dataset[:train_size]['Close'], mode='lines', name='Train Close Price'))

# Thêm trace cho giá đóng cửa thực tế của tập kiểm tra
fig.add_trace(go.Scatter(x=valid_data.index, y=valid_data['Close'], mode='lines', name='Test Close Price'))

# Thêm trace cho giá đóng cửa dự đoán của tập kiểm tra
fig.add_trace(go.Scatter(x=valid_data.index, y=valid_data['Predictions'], mode='lines', name='Predicted Close Price'))

# Thiết lập tiêu đề và trục
fig.update_layout(
    title='Biểu đồ Dự đoán giá đóng cửa chứng khoán bằng XGBoost',
    xaxis_title='Ngày',
    yaxis_title='Giá Đóng Cửa (Close)',
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    )
)

# Hiển thị biểu đồ
fig.show()


In [50]:
# Đánh giá mô hình
mae = mean_absolute_error(valid_data['Close'], valid_data['Predictions'])
mse = mean_squared_error(valid_data['Close'], valid_data['Predictions'])
rmse = np.sqrt(mse)
r2 = r2_score(valid_data['Close'], valid_data['Predictions'])

print(f"Mean Absolute Error (MAE): {mae}")
print(f"Mean Squared Error (MSE): {mse}")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"R-squared (R²): {r2}")


Mean Absolute Error (MAE): 2401.8788039028386
Mean Squared Error (MSE): 13507435.881705053
Root Mean Squared Error (RMSE): 3675.2463702050036
R-squared (R²): 0.9426838569800213


# KẾT LUẬN: Các chỉ số đánh giá cho thấy GRU có MAE, MSE, RMSE thấp nhất nhưng R² cao nhất, thể hiện đây là mô hình dự báo tốt nhất trong ba mô hình cho dữ liệu. LSTM là lựa chọn thứ hai, tốt hơn XGBoost nhưng kém hơn GRU. XGBoost có hiệu suất thấp nhất trong trường hợp này, không phải là lựa chọn tốt cho dự báo giá chứng khoán trên chuỗi thời gian này.